## Wildfire dataset

National Intragency Fire Center:
- Historical wildfires: https://data-nifc.opendata.arcgis.com/datasets/nifc::wfigs-wildland-fire-locations-full-history/about

In [1]:
! pip install geopandas
! pip install fiona
! pip install folium
! pip install mapclassify
! pip install contextily
! pip install sentinelhub

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ML-Climate-Predicting-Wildfires/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### From fire boundary shapes, identify rectangular regions to target for Sentinel images

In [3]:
import fiona 
import geopandas as gpd
import pandas as pd
import contextily as cx
import datetime

# https://data-nifc.opendata.arcgis.com/datasets/nifc::wfigs-wildland-fire-locations-full-history/explore?location=-0.208721%2C121.952724%2C0.74
path_to_data = "data/RawData/NIFC_Wildland_Fire_Perimeters.gdb"
fires_gdf = gpd.read_file(path_to_data)
fires_gdf

,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,...,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,geometry
0,None,CERTIFIED,50.64,2020-08-06T23:13:07+00:00,2020-08-06T23:13:24+00:00,50.6,20.0,MTMCC,NaN,None,...,None,None,2020-08-06T19:50:29+00:00,2020-08-12T20:46:01+00:00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,None,None,POINT (-104.45751 45.78504)
1,None,DEFAULT,NaN,None,None,NaN,0.1,CALACC,NaN,None,...,None,None,2020-02-28T20:52:36+00:00,2020-02-28T20:52:36+00:00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,None,None,POINT (-118.18071 33.80898)
2,None,DEFAULT,NaN,2017-10-18T00:30:00+00:00,2017-10-18T00:35:00+00:00,50.0,50.0,MTKIC,NaN,None,...,None,None,2017-10-18T13:46:40+00:00,2017-11-09T22:08:19+00:00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,None,None,POINT (-114.83541 48.07395)
3,None,DEFAULT,NaN,None,None,NaN,NaN,CAMVIC,NaN,None,...,None,None,2019-07-01T20:10:12+00:00,2019-07-01T20:10:12+00:00,IRWIN,{5DF06F41-9948-49D3-B00A-2D3A1D1049C5},NaN,None,None,POINT (-117.15390 33.17639)
4,None,DEFAULT,NaN,None,None,NaN,NaN,None,NaN,None,...,None,None,2016-06-20T22:39:02+00:00,2016-06-20T22:39:02+00:00,IRWIN,{F378818E-D541-4E0A-9A44-C81886C2B8B4},NaN,None,None,POINT (-121.10418 38.83473)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212233,None,DEFAULT,NaN,None,None,NaN,NaN,SDGPC,NaN,None,...,None,None,2022-03-21T13:24:39+00:00,2022-03-21T13:24:39+00:00,IRWIN,{C15A3B09-A767-40FB-8404-DFA2CBC0F42F},0.0,None,None,POINT (-99.45001 44.06140)
212234,None,DEFAULT,NaN,None,None,1.0,1.0,ILILC,NaN,None,...,None,None,2022-03-21T13:29:12+00:00,2022-03-21T13:46:28+00:00,IRWIN,{054AFEB9-7BAF-425E-B528-2E6C436547D0},0.0,None,None,POINT (-88.03056 41.37154)
212235,None,DEFAULT,NaN,None,None,1.0,1.0,ILILC,NaN,None,...,None,None,2022-03-21T13:36:34+00:00,2022-03-21T13:48:06+00:00,IRWIN,{9F4846CC-07EC-4D84-9D37-DD016F11A502},0.0,None,None,POINT (-88.15883 41.34809)
212236,None,DEFAULT,NaN,None,None,1.0,1.0,ILILC,NaN,None,...,None,None,2022-03-21T13:39:35+00:00,2022-03-21T13:48:28+00:00,IRWIN,{A2794C09-838A-4434-97F4-9E82289DDC18},0.0,None,None,POINT (-88.16037 41.35492)


In [4]:
fires_gdf['discovery_date'] = pd.to_datetime(fires_gdf['FireDiscoveryDateTime']).dt.tz_localize(None)
fires_gdf = fires_gdf[fires_gdf['discovery_date'] > datetime.datetime(year=2017,month=1,day=1)]

In [32]:
fires_gdf = fires_gdf.to_crs('NAD83')
proj_gdf = fires_gdf.to_crs('+proj=cea')
centroid = proj_gdf.centroid.to_crs('+proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ')
buffer = centroid.buffer(1000, cap_style=3)
fires_gdf['buffer_4326'] = buffer.to_crs('EPSG:4326')

In [33]:
columns =  ['discovery_date',
            'IncidentName',
            'FireCause',
            'CalculatedAcres',
            'ContainmentDateTime',
            'centroid',
            'UniqueFireIdentifier',
            'buffer_4326']
fires_df = fires_gdf[columns]

In [56]:
big_fires_gdf = fires_gdf[fires_gdf['DailyAcres']>100]
len(fires_gdf), len(big_fires_gdf)

(159446, 12225)

### Download Sentinel images for positive training points

In [34]:
from sentinelhub import SHConfig, SentinelHubCatalog, SentinelHubDownloadClient

config = SHConfig()
#stoner
config.sh_client_id = 'db3185e7-7d29-447b-91e6-0f5e31b71e42'
config.sh_client_secret = 'yM@G{l+)G4pN8>!u1h&R@.]jrP#%i3wa98G]}o>k'
config.save()
catalog = SentinelHubCatalog(config=config)
catalog.get_info()
client = SentinelHubDownloadClient(config=config)

In [35]:
%matplotlib inline

import datetime as dt

import numpy as np
import matplotlib.pyplot as plt

from sentinelhub import SHConfig, BBox, CRS, DataCollection,\
    SentinelHubRequest, filter_times, bbox_to_dimensions, \
    MimeType, SentinelHubDownloadClient


In [46]:
from datetime import timedelta, datetime

def get_bbox(fire):
    return fire['buffer_4326'].bounds

def get_time_interval(fire, t_pre, t_post):
    date_f = fire['discovery_date']
    delta_pre = timedelta(days=t_pre)
    delta_post = timedelta(days=t_post)
    date_i = date_f - delta_pre
    datestr_i = datetime.strftime(date_i, "%Y-%m-%d")
    date_f = date_f + delta_post
    datestr_f = datetime.strftime(date_f, "%Y-%m-%d")

    return [datestr_i, datestr_f]

evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12"],
                units: "DN"
            }],
            output: {
                bands: 12,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B11,
                sample.B12];
    }
"""

def get_percent_missing(image):
    return len(image[image==0]) / len(image[image>=0])

def get_best_timestamp(fire_bbox, fire_time_interval):
    results = []
    unique_acquisitions = None
    # Find valid dates in catalog
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=fire_bbox,
        time=fire_time_interval,
        query={
            "eo:cloud_cover": {
                "lt": 5
            }
        },
        fields={
            "include": [
                "id",
                "properties.datetime",
                "properties.eo:cloud_cover"
            ],
            "exclude": []
        }
    )
    results = list(search_iterator)
    if results:
      cloud_covers = [r['properties']['eo:cloud_cover'] for r in results]
      timestamps = search_iterator.get_timestamps()
      results_info = list(zip(timestamps, cloud_covers))
      results_info.sort(key = lambda x: x[1]) 
      time_difference = dt.timedelta(seconds=0)
      for timestamp, cc in results_info:
          request = SentinelHubRequest(
              evalscript=evalscript_all_bands,
              input_data=[
                  SentinelHubRequest.input_data(
                      data_collection=DataCollection.SENTINEL2_L2A,
                      time_interval=(timestamp - time_difference, timestamp + time_difference)
                  )
              ],
              responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
              bbox=fire_bbox,
              size=bbox_to_dimensions(fire_bbox, 20),
              config=config
          )
          data = request.get_data()
          if get_percent_missing(data[0]) < 0.2:
            return timestamp, data[0]
    return None, None

In [61]:
import json 

fire_image_dir = "data/TrainingData/Fire-Positive-5mo/"
download_log_file = "{}download_log.json".format(fire_image_dir)

if os.path.exists(download_log_file):
  with open(download_log_file) as f:
    selected_image_info = json.load(f)
else:
  selected_image_info = {}

len(selected_image_info)

4576

In [60]:
twoweek_days = 14
for index, fire in big_fires_gdf.iterrows():
    print(index/max(big_fires_gdf.index)*100, '%')
    fire_id = fire['UniqueFireIdentifier']
    if not fire_id in selected_image_info:
        print(fire_id)
        fire_bbox = BBox(get_bbox(fire), crs=CRS.WGS84)
        fire_time_interval = get_time_interval(fire, t_pre=3*twoweek_days, t_post=-2*twoweek_days)
        first_timestamp, first_image = get_best_timestamp(fire_bbox, fire_time_interval)
        if first_timestamp:
          timestamps = [first_timestamp]
          images = [first_image]
          valid_timestamps = 1
          for i in range(1,6):
            fire_time_interval = get_time_interval(fire, t_pre=(i+3)*twoweek_days, t_post=-(i+2)*twoweek_days)
            timestamp, image = get_best_timestamp(fire_bbox, fire_time_interval)
            valid_timestamps += 1 if timestamp else 0
            timestamps.append(timestamp if timestamp else timestamps[-1])
            images.append(image if image is not None else images[-1])
          if valid_timestamps>=3:
            selected_image_info[fire_id] = [str(timestamp.date()) for timestamp in timestamps]
            for i, image in enumerate(images):
                print(fire['discovery_date'], '-->', timestamps[i])
                # True color
                # plt.figure(figsize=(9,9))
                # plt.imshow(np.clip(image[:,:,1:4]*.5/1000, 0, 255))
                # plt.show()
                with open(fire_image_dir + "{}_{}.npy".format(fire_id, i), 'wb') as f:
                    np.save(f, image)
          else:
            print('Insufficient images in timeframe', valid_timestamps)
        else:
            print('No images in timeframe')
            selected_image_info[fire_id] = []
        with open(download_log_file, 'w') as f:
            f.write(json.dumps(selected_image_info))

0.002827028274994464 %
0.00848108482498339 %
0.02261622619995571 %
0.024500911716618688 %
0.03345316792076782 %
0.03910722447075675 %
0.040049567229088236 %
0.04664596653740865 %
0.05606939412072353 %
2017-NVWID-020138
Insufficient images in timeframe 1
0.06360813618737543 %
0.07774327756234775 %
0.09894598962480623 %
0.10978293134561834 %
0.11826401617060173 %
0.12014870168726471 %
0.12297572996225917 %
0.12627392961641937 %
0.14747664167887786 %
0.1578424120205242 %
0.15878475477885573 %
0.16255412581218165 %
0.1715063820163308 %
0.18564152339130313 %
0.20354603579960137 %
0.20731540683292735 %
0.2219217195870654 %
0.22663343337872283 %
0.2322874899287118 %
0.2369992037203692 %
0.24171091751202667 %
0.24359560302868963 %
0.292597426461927 %
0.29542445473692147 %
0.30202085404524187 %
0.3076749105952308 %
0.3227523947285346 %
0.3359451933451755 %
0.3430127640326616 %
0.3439551067909931 %
0.3463109636868218 %
0.3557343912701367 %
0.3689271898867775 %
0.3731677322992692 %
2017-ORUPF-000

KeyboardInterrupt: ignored